In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
pd.set_option('display.max_columns', None)

In [2]:
odds = pd.read_excel('data/nhl odds 2021.xlsx')

In [4]:
odds.head()

,Date,Rot,VH,Team,1st,2nd,3rd,Final,Open,Close,PuckLine,Unnamed: 11,OpenOU,Unnamed: 13,CloseOU,Unnamed: 15
0,113,41,V,Pittsburgh,1,1,1,3,-110,-115,1.5,-310,6.0,-110,6.0,105
1,113,42,H,Philadelphia,2,1,3,6,100,-105,-1.5,260,6.0,-110,6.0,-125
2,113,43,V,Montreal,2,1,1,0,132,120,1.5,-220,6.0,-110,6.0,-110
3,113,44,H,Toronto,1,2,1,1,-145,-140,-1.5,195,6.0,-110,6.0,-110
4,113,45,V,Chicago,0,0,1,1,230,210,1.5,-125,6.5,-110,6.5,-110


In [5]:
team_conversion =   { 'Anaheim': 'ANA', 
                      'Arizona' :'ARI', 
                      'Boston': 'BOS', 
                      'Buffalo':'BUF',
                      'Calgary': 'CGY', 
                      'Carolina': 'CAR', 
                      'Chicago': 'CHI', 
                      'Colorado': 'COL',
                     'Columbus': 'CBJ',
                     'Dallas': 'DAL',
                     'Detroit': 'DET',
                     'Edmonton': 'EDM',
                     'Florida': 'FLA',
                     'LosAngeles': 'L.A.',
                     'Minnesota': 'MIN',
                     'Montreal': 'MTL',
                     'Nashville': 'NSH',
                     'NewJersey': 'N.J.',
                     'NYIslanders': 'NYI',
                     'NYRangers': 'NYR',
                     'Ottawa': 'OTT',
                     'Philadelphia': 'PHI',
                     'Pittsburgh': 'PIT',
                     'SanJose': 'S.J.',
                     'St.Louis': 'STL',
                     'TampaBay': 'T.B.',
                     'Toronto': 'TOR',
                     'Vancouver': 'VAN',
                      'Vegas':'VGK',
                     'Washington': 'WSH',
                     'Winnipeg': 'WPG'}

In [6]:
odds = odds.replace({'Team': team_conversion})

In [15]:
odds['Month'] = odds['Date'].apply(lambda x: str(x)[0])
odds['Day'] = odds['Date'].apply(lambda x: str(x)[1:])
odds['Year'] = 2021

In [16]:
odds.head()

,Date,Rot,VH,Team,1st,2nd,3rd,Final,Open,Close,PuckLine,Unnamed: 11,OpenOU,Unnamed: 13,CloseOU,Unnamed: 15,Day,Month,Year
0,113,41,V,PIT,1,1,1,3,-110,-115,1.5,-310,6.0,-110,6.0,105,13,1,2021
1,113,42,H,PHI,2,1,3,6,100,-105,-1.5,260,6.0,-110,6.0,-125,13,1,2021
2,113,43,V,MTL,2,1,1,0,132,120,1.5,-220,6.0,-110,6.0,-110,13,1,2021
3,113,44,H,TOR,1,2,1,1,-145,-140,-1.5,195,6.0,-110,6.0,-110,13,1,2021
4,113,45,V,CHI,0,0,1,1,230,210,1.5,-125,6.5,-110,6.5,-110,13,1,2021


In [19]:
odds['Datetime'] = pd.to_datetime(odds[['Day','Month','Year']]
                   .astype(str).apply(' '.join, 1))

odds['Team_Key'] = odds['Team'].astype(str)+'_'+odds['Datetime'].astype(str)

In [31]:
odds['Implied_odds'] = np.where(odds['Close'] < 0, (odds['Close']*-1)/((odds['Close']*-1)+100) , 100/(odds['Close']+100))

In [32]:
odds.head(6)

,Date,Rot,VH,Team,1st,2nd,3rd,Final,Open,Close,PuckLine,Unnamed: 11,OpenOU,Unnamed: 13,CloseOU,Unnamed: 15,Day,Month,Year,Datetime,Team_Key,Implied_odds
0,113,41,V,PIT,1,1,1,3,-110,-115,1.5,-310,6.0,-110,6.0,105,13,1,2021,2021-01-13,PIT_2021-01-13,0.534884
1,113,42,H,PHI,2,1,3,6,100,-105,-1.5,260,6.0,-110,6.0,-125,13,1,2021,2021-01-13,PHI_2021-01-13,0.512195
2,113,43,V,MTL,2,1,1,0,132,120,1.5,-220,6.0,-110,6.0,-110,13,1,2021,2021-01-13,MTL_2021-01-13,0.454545
3,113,44,H,TOR,1,2,1,1,-145,-140,-1.5,195,6.0,-110,6.0,-110,13,1,2021,2021-01-13,TOR_2021-01-13,0.583333
4,113,45,V,CHI,0,0,1,1,230,210,1.5,-125,6.5,-110,6.5,-110,13,1,2021,2021-01-13,CHI_2021-01-13,0.322581
5,113,46,H,T.B.,3,1,1,5,-260,-250,-1.5,105,6.5,-110,6.5,-110,13,1,2021,2021-01-13,T.B._2021-01-13,0.714286


In [29]:
210/310

0.6774193548387096